In [1]:
import requests
import sys
import numpy as np
import time
time.sleep(1)
page = requests.get("https://www.boxofficemojo.com/year/2019/")
page.status_code

200

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo">
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta charset="utf-8"/>
  <title dir="ltr">
   Domestic Box Office For 2019 - Box Office Mojo
  </title>
  <meta content="Domestic Box Office For 2019" name="title"/>
  <meta content="Box Office Mojo" property="og:site_name"/>
  <meta content="telephone=no" name="format-detection"/>
  <link href="https://m.media-amazon.com/images/G/01/boxofficemojo/v2/favicon._CB448965889_.ico" rel="icon" type="image/x-icon"/>
  <link href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|012LjolmrML.css,51AZ-Jz5kmL.css,51IB+wfP8qL.css,01evdoiemkL.css,01K+Ps1DeEL.css,01Vctty9pOL.css,314djKvMsUL.css,01ZTetsDh7L.css,11cMnOipjJL.css,01pbA9Lg3yL.css,21LK7jaicML.css,11L58Qpo0GL.css,21kyTi1FabL.css,01ruG+gDPFL.css,01YhS3Cs-hL.css,21GwE3cR-yL.css,11KLBtpWIAL.

In [3]:
links = soup.select(".a-text-left.mojo-field-type-release.mojo-cell-wide a")
pages = []
for i in links:
    pages.append(requests.get("https://www.boxofficemojo.com" + i["href"]))
    time.sleep(np.random.random())
    pages[-1]

KeyboardInterrupt: 